In [1]:
# Importing all the necessary libraries
import pandas as pd
import zipfile
import os
import json
import re
from collections import Counter
import matplotlib.pyplot as plt
import ollama

In [3]:
#Reading the json files of our sampled job listings. Due to the large size of the files, we will read them year by year. 
# The code below is an example using df_16 as the dataframe for the year 2016. The same code will be used for the other years.
df_16 = pd.read_json('sampled_job_listings_2016.json')




The joblistings with a word count below a certain number don't seem to give us much to work with. We'll remove them.

In [4]:
 # Calculate word counts for each row in the 'HTML_Text' column
df_16['word_count'] = df_16['HTML_Text'].apply(lambda x: len(x.split()))

# Generate statistics on the word counts
stats = df_16['word_count'].describe()

# Print the statistics
print(stats)

count    1000.000000
mean      792.253000
std       230.635057
min       400.000000
25%       636.750000
50%       769.000000
75%       916.500000
max      2145.000000
Name: word_count, dtype: float64


In [5]:
# We define a threshold for minimum word count
min_word_count = 400

# Filter the original dataframe in place, only keeping rows meeting the word count requirement
df_16 = df_16[df_16['HTML_Text'].apply(lambda x: len(x.split()) > min_word_count)].copy()

df_16.shape

(999, 9)

In [6]:
# Function to clean trailing indicators
def clean_trailing_indicators(area):
    if pd.isna(area):
        return None
    # Remove trailing indicators like 'o', 'c', 'SØ', 'N', 'K'
    area = re.sub(r'\s+[a-zA-ZÆØÅæøå]+$', '', area.strip())
    return area

# Apply the cleaning function to the 'Area' column
df_16['Area'] = df_16['Area'].apply(clean_trailing_indicators)

In [7]:
# Display all unique values in the 'Area' column
unique_areas = df_16['Area'].unique()
print("Unique areas:")
for area in unique_areas:
    print(area)

Unique areas:
Hørsholm or
Vedbæk
Silkeborg
Frederikshavn
Aalborg
Ballerup
København
Skærbæk
Søborg
Hvidovre
Region
Valby
Storkøbenhavn
Copenhagen
Sverige
Frankrig
Aarhus
Fredriksberg
Fredericia
Kolding
Hørsholm
Glostrup
Esbjerg
Randers
Tranbjerg J.
Danmark, Storkøbenhavn,
Frederiksberg
Holstebro
Smørum
Høje
Skåne
Skanderborg
Hillerød
København eller
Bagsværd
Vallensbæk
Værløse
Kuwait
Danmark,
Danmark, Region
Brande
Brønshøj
Kgs.
Brøndby
Viby J.
Odense
Vejle
Risskov
Lyngby
Nykøbing
Mozambique
Albertslund
Vojens
Litauen,
Taastrup
Bryrup
Litauen
Bjerringbro
Måløv
Billund
Aarhus, Kolding
Udlandet
Ringsted
Rødovre
Charlottenlund
Tyskland
Hedensted
Lystrup
København/Aalborg
Herlev
Skovlunde
Allerød
Humlebæk
Nærum
Højbjerg
København, Aarhus
Sorø
Århus/Brande
Vassingerød
Virum
Århus
Broager
Greve eller
Oslo or
Brande and
Skjern
Oksbøl
Luxembourg
Amsterdam
Ikast
Helsingør
Nordborg
Lynge
Albertslund,
Sydjylland
Ryslinge
Horsens
Hellerup
Vanløse
Grindsted
Gentofte
Ishøj
Viby
Slagelse
Støvring
Mid

In [8]:
# Mapping of Danish to English city names
city_name_mapping = {
    'København': 'Copenhagen',
    'Århus': 'Aarhus',
    'Helsingør': 'Elsinore'
}

# Replace Danish names with their English equivalents
df_16['Area'] = df_16['Area'].replace(city_name_mapping)

In [9]:
cities = [
    "Copenhagen", "Aarhus", "Odense", "Aalborg", "Esbjerg", "Randers", "Kolding", "Horsens", "Vejle", "Roskilde",
    "Herning", "Hørsholm", "Silkeborg", "Næstved", "Fredericia", "Viborg", "Køge", "Holstebro", "Taastrup", "Slagelse",
    "Hillerød", "Sønderborg", "Svendborg", "Hjørring", "Holbæk", "Frederikshavn", "Nørresundby", "Ringsted", "Haderslev",
    "Skive", "Ølstykke-Stenløse", "Nykøbing Falster", "Greve Strand", "Kalundborg", "Ballerup", "Rødovre", "Lyngby",
    "Albertslund", "Hvidovre", "Glostrup", "Ishøj", "Birkerød", "Farum", "Frederikssund", "Brøndby Strand",
    "Skanderborg", "Hedensted", "Frederiksværk", "Lillerød", "Solrød Strand"
]

# Filter the DataFrame to only include rows where Area is in the cities list
df_16 = df_16[df_16['Area'].isin(cities)]

# Display the filtered DataFrame
print("Filtered DataFrame:")
print(df_16.shape)

Filtered DataFrame:
(655, 9)


In [10]:
# Filter the DataFrame to only include rows where Area is in the cities list
df_16 = df_16[df_16['Area'].isin(cities)]

# Display the filtered DataFrame
print("Filtered DataFrame:")
print(df_16.shape)

Filtered DataFrame:
(655, 9)


In [11]:
# Sets up environment variables and starts the Ollama server
import os
import threading
import subprocess

def start_ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])

ollama_thread = threading.Thread(target=start_ollama)
ollama_thread.start()

python(98479) MallocStackLogging: can't turn off malloc stack logging because it was not enabled.


2025/01/20 12:53:26 routes.go:1197: INFO server config env="map[HTTPS_PROXY: HTTP_PROXY: NO_PROXY: OLLAMA_DEBUG:false OLLAMA_FLASH_ATTENTION:false OLLAMA_GPU_OVERHEAD:0 OLLAMA_HOST:http://0.0.0.0:11434 OLLAMA_KEEP_ALIVE:5m0s OLLAMA_LLM_LIBRARY: OLLAMA_LOAD_TIMEOUT:5m0s OLLAMA_MAX_LOADED_MODELS:0 OLLAMA_MAX_QUEUE:512 OLLAMA_MODELS:/Users/anders-macbookpro/.ollama/models OLLAMA_MULTIUSER_CACHE:false OLLAMA_NOHISTORY:false OLLAMA_NOPRUNE:false OLLAMA_NUM_PARALLEL:0 OLLAMA_ORIGINS:[* http://localhost https://localhost http://localhost:* https://localhost:* http://127.0.0.1 https://127.0.0.1 http://127.0.0.1:* https://127.0.0.1:* http://0.0.0.0 https://0.0.0.0 http://0.0.0.0:* https://0.0.0.0:* app://* file://* tauri://* vscode-webview://*] OLLAMA_SCHED_SPREAD:false OLLAMA_TMPDIR: http_proxy: https_proxy: no_proxy:]"
time=2025-01-20T12:53:26.979+01:00 level=INFO source=images.go:753 msg="total blobs: 17"
time=2025-01-20T12:53:26.980+01:00 level=INFO source=images.go:760 msg="total unused bl

In [12]:
SYSTEM_PROMPT = """

Extract relevant skills from a job listing, separating them into **primary IT-related technical skills** and **secondary (soft/general) skills**.

### Primary Skills
Focus on IT-specific technical skills such as:
- Programming languages, frameworks, tools, platforms, and technologies.
- Examples: Python, React, Docker, AWS, SQL, Kubernetes.

### Secondary Skills
Focus on non-technical competencies, including:
- Communication, leadership, problem-solving, teamwork, time management.

---

### Rules:
- Categorize skills into **primary** (IT-related) and **secondary** (soft skills).
- Split compound terms into separate skills (e.g., "Python and Java" → ["Python", "Java"]).
- Exclude vague, repetitive, or non-actionable terms.
- Translate outputs to **English only**, regardless of input language.
- Ensure outputs are clear, specific, and concise.

---

### Output Format:
```json
{
  "skills": {
    "primary": ["Skill 1", "Skill 2", "..."],
    "secondary": ["Skill A", "Skill B", "..."]
  }
}

"""

In [13]:
# Function to extract skills (primary and secondary) from our job listings. We use the model llama3.1:8b. Below is an example for of code used to extract skills from 2016 job listings.
def extract_skills(html_text, row_id):
    prompt = f"""
    Extract the relevant skills from the following job listing, categorizing them into **primary (IT-related technical skills)** and **secondary (soft/general skills)**.

    ### Skills Categories:
    1. **Primary Skills:** IT-related technical skills (e.g., Python, AWS, Kubernetes, Docker, SQL, React, etc.).
    2. **Secondary Skills:** Soft skills and traits (e.g., Communication, Teamwork, Problem-Solving, Leadership).

    ### Rules:
    - Split combined skills into distinct terms (e.g., "Python and Java" → ["Python", "Java"]).
    - Focus on actionable, specific, and clearly stated skills.
    - Outputs must be in **clear and concise English only**, regardless of the input language.
    - If no skills are present, return empty lists for both categories.
    - Translate all extracted skills into **English** if the input text is in Danish.

    Provide the results in this JSON format:
    {{
      "skills": {{
        "primary": ["Skill 1", "Skill 2", "..."],
        "secondary": ["Skill A", "Skill B", "..."]
      }}
    }}

    Job Listing Text:
    {html_text}
    """
    try:
        response = ollama.chat(
            model='llama3.1:8b',
            messages=[
                {'role': 'system', 'content': SYSTEM_PROMPT},
                {'role': 'user', 'content': prompt},
            ],
            format='json',
            options={"temperature": 0.1}
        )
        return json.loads(response['message']['content'])
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Error parsing JSON for Row {row_id}: {e}")
        return {"skills": {"primary": [], "secondary": []}}

# Function to refine extracted skills
def refine_skills(extracted_skills, row_id):
    refinement_prompt = f"""
    Refine the following extracted skills, ensuring accuracy and clear categorization:

    Extracted Skills:
    {json.dumps(extracted_skills, indent=2)}

    ### Refined Categories:
    1. **Primary Skills:** IT-related technical skills only.
    2. **Secondary Skills:** Clearly defined soft skills.

    ### Rules:
    - Keep skills actionable, specific, and IT-relevant for "primary."
    - Use concise and clear English.
    - Outputs must be in **clear and concise English.**
    - Translate all extracted skills into **English** if the input text is in Danish.

    Provide the results in this JSON format:
    {{
      "skills": {{
        "primary": ["Skill 1", "Skill 2", "..."],
        "secondary": ["Skill A", "Skill B", "..."]
      }}
    }}
    """
    try:
        response = ollama.chat(
            model='llama3.1:8b',
            messages=[
                {'role': 'system', 'content': SYSTEM_PROMPT},
                {'role': 'user', 'content': refinement_prompt},
            ],
            format='json',
            options={"temperature": 0.1}
        )
        return json.loads(response['message']['content'])
    except (json.JSONDecodeError, KeyError) as e:
        print(f"Error refining skills for Row {row_id}: {e}")
        return {"skills": {"primary": [], "secondary": []}}

# Initialize the DataFrame with the columns for Primary_Skills and Secondary_Skills
if 'Primary_Skills' not in df_16.columns:
    df_16['Primary_Skills'] = None
if 'Secondary_Skills' not in df_16.columns:
    df_16['Secondary_Skills'] = None

# Process job listings in batches of 10 rows
batch_size = 10
for batch_start in range(0, len(df_16), batch_size):
    batch_end = batch_start + batch_size
    print(f"Processing batch: Rows {batch_start} to {min(batch_end, len(df_16)) - 1}")
    
    for index, html_text in df_16['HTML_Text'][batch_start:batch_end].items():
        print(f"Processing index {index}")  # Log the index being processed

        # Extract skills
        extracted = extract_skills(html_text, row_id=index)
        print(f"Extracted Skills from Row {index}: {extracted}")

        # Refine extracted skills
        refined = refine_skills(extracted, row_id=index)
        print(f"Refined Skills for Row {index}: {refined}")

        # Ensure refined skills structure is correct
        if isinstance(refined, dict) and 'skills' in refined and isinstance(refined['skills'], dict):
            df_16.at[index, 'Primary_Skills'] = refined['skills'].get('primary', [])
            df_16.at[index, 'Secondary_Skills'] = refined['skills'].get('secondary', [])
        else:
            print(f"Invalid structure for refined skills at Row {index}. Defaulting to empty lists.")
            df_16.at[index, 'Primary_Skills'] = []
            df_16.at[index, 'Secondary_Skills'] = []

# Display the updated DataFrame
print(df_16[['Primary_Skills', 'Secondary_Skills']])

# Save the updated DataFrame to a CSV file
output_file = 'updated_job_listings_with_skills_2016.csv'
try:
    df_16.to_csv(output_file, index=False, encoding='utf-8')
    print(f"Updated job listings saved to {output_file}.")
except IOError as e:
    print(f"Error saving file: {e}")

Processing batch: Rows 0 to 9
Processing index 2


KeyboardInterrupt: 